In [2]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
import numpy as np
def json_prettify(json_res):
    return json.dumps(json_res.json(), indent=2)

def get_auth():
    url = 'http://172.18.65.246/zabbix/api_jsonrpc.php'

    headers = {
        'Content-Type': 'application/json-rpc'
    }
    auth_query = {
        "jsonrpc": "2.0",
        "method": "user.login",
        "params": {
            "user": "QAC",
            "password": "QAC12345"
        },
        "id": 1
    }


    response = requests.post(url, headers=headers, data=json.dumps(auth_query)).json()
    return response['result']
auth = get_auth()

# def to_timestamp(unix_timestamp):
#     # Convert Unix timestamp to a datetime object
#     timestamp = datetime.utcfromtimestamp(int(unix_timestamp))
#     timestamp = timestamp + timedelta(hours=7)
#     # Format the datetime object as a human-readable timestamp
#     formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:00')

#     return formatted_timestamp

def to_timestamp(unix_timestamp):
    # Convert Unix timestamp to a datetime object
    timestamp = datetime.utcfromtimestamp(int(unix_timestamp))
    timestamp = timestamp + timedelta(hours=7)
    # Format the datetime object as a human-readable timestamp
    formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:00')

    return formatted_timestamp

In [3]:
def get_data(auth, history, time_start, time_till, item_id):
    time_start = int(datetime.strptime(time_start, "%Y-%m-%d %H:%M:%S").timestamp())
    time_till = int(datetime.strptime(time_till, "%Y-%m-%d %H:%M:%S").timestamp())

    url = 'http://172.18.65.246/zabbix/api_jsonrpc.php'
    headers = {
        'Content-Type': 'application/json-rpc',
    }

    data_query = {
            "jsonrpc": "2.0",
            "method": "history.get",
            "params": {
                "output": "extend",
                "history": history,
                "itemids": item_id,
                "sortfield": "clock",
                "sortorder": "DESC",
                "time_from": time_start,
                "time_till": time_till,
                "limit": 50000
            },
            "auth": auth,
            "id": 1
        }

    response = requests.post(url, headers=headers, data=json.dumps(data_query)).json()
    
    df = pd.DataFrame(response['result'])

    return df

In [4]:
def date_to_unix(start_date, end_date):
    # Convert datetime objects to Unix timestamps
    start_date = int(start_date.timestamp())
    end_date = int(end_date.timestamp())

    # Add 7 hours to the Unix timestamps
    start_unix = start_date + 7 * 3600
    end_unix = end_date + 7 * 3600

    return start_unix, end_unix

def fill_null(dataframe):
    for i in range(len(dataframe)):
        for column in dataframe[['temp','hum']]:
            if pd.isnull(dataframe.at[i, column]):
                if i == 0:
                    num = 0
                    while num < len(dataframe)-1 and pd.isnull(dataframe.at[num, column]):                    
                        num +=1
                    next_value = dataframe.at[num, column]
                    dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = next_value
                    dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
                else:
                    prev_value = dataframe.at[i - 1, column]
                    dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = prev_value
                    dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
    return dataframe

In [5]:
# LANTAI 3
df_rekap_3 = pd.read_excel('rekap DCI Lantai 3.xlsx', sheet_name='Sheet7')
# a = df_rekap_3.to_json(path_or_buf="./rekap DCI Lantai3.json",orient = "records")
df_rekap_3 = df_rekap_3[:]
df_rekap_3[['hum_id','temp_id']] = df_rekap_3[['hum_id','temp_id']].fillna(0).astype('int')

In [5]:
df_rekap_3[109:]

,hostname,filename,hum_id,temp_id,hum_hist,temp_hist
109,J10_DC3_GTI,J 10_DC3,31230,31231,0,0
110,J13 DC3 GTI,J 13_DC3,30468,30470,0,0
111,J18 DC3 GTI,J 18_DC3,29779,29781,0,0
112,J26_DC3_GTI,J 26_DC3,31670,31672,0,0
113,J34 DC3 GTI,J 34_DC3,30040,30042,0,0
...,...,...,...,...,...,...
171,M61 DC3 GTI,M 61_DC3,30184,30186,0,0
172,N61 DC3 GTI,N 61_DC3,40517,40519,0,0
173,L66 DC3 GTI,L 66_DC3,33986,33988,0,0
174,H45 DC3 GTI,H 45_DC3,31497,31498,0,0


In [6]:
# LANTAI 3

time_start = "2024-04-01 00:00:00"
time_till = "2024-04-30 23:59:59"

base_dir = './result/Lantai 3/DCI - Lantai 3 (Apr 2024)/'

timestamp_list = pd.date_range(start=time_start, end=time_till, freq='1T')
summary = pd.DataFrame(columns=['hostname','temp_id', 'total_temp','hum_id','total_hum'])

for index, data in df_rekap_3.iterrows():    
    hostname = data['hostname']
    filename= data['filename']
    host_id = ['']
    hum_id = data['hum_id']
    temp_id = data['temp_id']

    raw_temp = pd.DataFrame(columns=['itemid','clock','value','ns'])
    raw_hum = pd.DataFrame(columns=['itemid','clock','value','ns'])

    raw_temp_data = get_data(auth, data['temp_hist'], time_start, time_till, temp_id)
    raw_hum_data = get_data(auth, data['hum_hist'],time_start, time_till, hum_id)


    raw_temp = pd.concat([raw_temp, raw_temp_data], axis=0)
    raw_hum = pd.concat([raw_hum, raw_hum_data], axis=0)
    raw_temp['date'] = raw_temp['clock'].apply(to_timestamp)
    raw_hum['date'] = raw_hum['clock'].apply(to_timestamp)

    raw_temp = raw_temp.groupby(by='date').max().reset_index()
    raw_hum = raw_hum.groupby(by='date').max().reset_index()


    result = pd.DataFrame(columns=['hostname','date'])
    result['date'] = timestamp_list
    result['date'] = result['date'].astype('string')
    # result['date'] = pd.concat([raw_temp['date'], raw_hum['date']]).sort_values(ascending=True).unique()
    result = pd.merge(result, raw_temp[['date','value']].rename(columns={'value':'temp'}), on=['date'], how='left')
    result = pd.merge(result, raw_hum[['date','value']].rename(columns={'value':'hum'}), on=['date'], how='left')

    result['hostname'] = hostname
    # result[['temp','hum']] = result[['temp','hum']].astype('float')
    result[['temp_null','hum_null']] = ''
    
    final_data = fill_null(result)
    final_data.to_csv(str(base_dir+filename+'.csv'), index=False)
    data = {'hostname': hostname,
        'temp_id': temp_id,
        'total_temp':len(raw_temp),
        'hum_id': hum_id,
        'total_hum': len(raw_hum),
    }
    summary.loc[len(summary)] = data
    print(hostname)
    
summary.to_excel(str(base_dir+'summary4.xlsx'), index=False)

E10 DC3 GTI
E13 DC3 GTI
E18 DC3 GTI
E26_DC3_GTI
E29 DC3 GTI
E34 DC3 GTI
E37 DC3 GTI
E42 DC3 GTI
E45 DC3 GTI
E5 DC3 GTI
E53 DC3 GTI
E61 DC3 GTI
E66 DC3 GTI
F18 DC3 GTI
F21_DC3_GTI
F26 DC3 GTI
F29 DC3 GTI
F34 DC3 GTI
F37 DC3 GTI
F42 DC3 GTI
F45 DC3 GTI
F53 DC3 GTI
F61 DC3 GTI
F66 DC3 GTI
G10 DC3 GTI
G13 DC3 GTI
G18 DC3 GTI
G21_DC3_GTI
G26 DC3 GTI
G29 DC3 GTI
G34 DC3 GTI
G37 DC3 GTI
G42 DC3 GTI
G45 DC3 GTI
G53 DC3 GTI
G61 DC3 GTI
G66 DC3 GTI
GD31.E21
GD31.F5
GD31.G5
GD31.J5
GD31.K5
GD31.L5
GD31.P5
GD31.M34
GD31.R45-1
GD41.E59
GD41.E62
GD41.F59
GD41.G59
GD41.G62
GD41.H59
GD41.H62
GD41.i59
GD41.i62
GD41.J59
GD41.J62
GD41.K59
GD41.K62
GD41.L62
GD41.M62
GD41.O59
GN31.D10
GN31.D13
GN31.D4
GN31.D5
GN31.D6
GN31.D8
GN31.D9
GN31.i10
GN31.i11
GN31.i12
GN31.i13
GN31.i3
GN31.i4
GN31.i6
GN31.i7
GN31.L3
GN31.L4
GN31.L6
GN41.C2
GN41.C3
GN41.C5
GN41.H10
GN41.H12
GN41.H2
GN41.H3
GN41.H6
GN41.H7
GN41.H8
H10_DC3_GTI
H13 DC3 GTI
H18 DC3 GTI
H29 DC3 GTI
H34 DC3 GTI
H37_DC3_GTI
H42 DC3 GTI
H53 DC3 GTI
I10 DC3 

In [ ]:
df_rekap_3[60:120]
J10_DC3_GTI

In [11]:
# LANTAI 4
df_rekap_4 = pd.read_excel('rekap DCI Lantai 4.xlsx', sheet_name='Sheet2')

df_rekap_4 = df_rekap_4[:]

df_rekap_4[['hum_id','temp_id']] = df_rekap_4[['hum_id','temp_id']].fillna(0).astype('int')

In [23]:
# LANTAI 4

time_start = "2024-03-01 00:00:00"
time_till = "2024-03-31 23:59:59"
base_dir = './result/Lantai 4/DCI - Lantai 4 (Mar 2024)/'


timestamp_list = pd.date_range(start=time_start, end=time_till, freq='1T')

# try:
#     summary = pd.read_excel(summary_path)
# except Exception as E:
#     summary = pd.DataFrame(columns=['hostname','total_temp', 'latest_temp_timestamp','total_hum','latest_hum_timestamp'])
summary = pd.DataFrame(columns=['hostname','temp_id', 'total_temp','hum_id','total_hum'])
# for index, data in df_rekap[:-1].iterrows():
for index, data in df_rekap_4[:].iterrows():    
    hostname = data['hostname']
    filename = data['filename']
    host_id = ['']
    hum_id = data['hum_id']
    temp_id = data['temp_id']

    raw_temp = pd.DataFrame(columns=['itemid','clock','value','ns'])
    raw_hum = pd.DataFrame(columns=['itemid','clock','value','ns'])

    raw_temp_data = get_data(auth, data['temp_hist'], time_start, time_till, temp_id)
    raw_hum_data = get_data(auth, data['hum_hist'],time_start, time_till, hum_id)


    raw_temp = pd.concat([raw_temp, raw_temp_data], axis=0)
    raw_hum = pd.concat([raw_hum, raw_hum_data], axis=0)
    raw_temp['date'] = raw_temp['clock'].apply(to_timestamp)
    raw_hum['date'] = raw_hum['clock'].apply(to_timestamp)

    raw_temp = raw_temp.groupby(by='date').max().reset_index()
    raw_hum = raw_hum.groupby(by='date').max().reset_index()


    result = pd.DataFrame(columns=['hostname','date'])
    result['date'] = timestamp_list
    result['date'] = result['date'].astype('string')
    # result['date'] = pd.concat([raw_temp['date'], raw_hum['date']]).sort_values(ascending=True).unique()
    result = pd.merge(result, raw_temp[['date','value']].rename(columns={'value':'temp'}), on=['date'], how='left')
    result = pd.merge(result, raw_hum[['date','value']].rename(columns={'value':'hum'}), on=['date'], how='left')

    result['hostname'] = hostname
    result[['temp','hum']] = result[['temp','hum']].astype('float')
    result[['temp_null','hum_null']] = ''
    
    final_data = fill_null(result)
    final_data.to_csv(str(base_dir+filename+'.csv'), index=False)
    data = {'hostname': hostname,
        'total_temp':len(raw_temp),
        'latest_temp_timestamp':raw_temp['clock'].min(),
        'total_hum': len(raw_hum),
        'latest_hum_timestamp': raw_hum['clock'].min()
    }
    summary.loc[len(summary)] = data
    print(hostname)

# summary[['latest_temp_timestamp','latest_hum_timestamp']] = summary[['latest_temp_timestamp','latest_hum_timestamp']].fillna(0).astype('int')
# summary['latest_temp_timestamp'] = summary['latest_temp_timestamp'].apply(to_timestamp)
# summary['latest_hum_timestamp'] = summary['latest_hum_timestamp'].apply(to_timestamp)

summary.to_excel(str(base_dir+'c.xlsx'), index=False)


GD41.E59
GD41.E62
GD41.F59
GD41.G59
GD41.G62
GD41.H59
GD41.H62
GD41.i59
GD41.i62
GD41.J59
GD41.J62
GD41.K59
GD41.K62
GD41.L62
GD41.M62


In [39]:
summary[['latest_temp_timestamp','latest_hum_timestamp']] = summary[['latest_temp_timestamp','latest_hum_timestamp']].fillna(0).astype('int')
summary['latest_temp_timestamp'] = summary['latest_temp_timestamp'].apply(to_timestamp)
summary['latest_hum_timestamp'] = summary['latest_hum_timestamp'].apply(to_timestamp)

summary.to_excel('./DCI - Lantai 4 (Nov 2023)/summary2.xlsx', index=False)


In [32]:
df_rekap_3[133:]

,hostname,hum_id,temp_id,hum_hist,temp_hist
133,H45 DC3 GTI,0,31498,3,0
134,L10_DC3_GTI,31255,31257,0,0
135,M10 DC3 GTI,30450,30452,0,0
136,O10 DC3 GTI,30453,30455,0,0
137,Q10 DC3 GTI,30459,30461,0,0
138,L13 DC3 GTI,31856,31857,3,3
139,M13 DC3 GTI,30480,30482,0,0
140,P13 DC3 GTI,31843,31844,3,3
141,Q13 DC3 GTI,31830,31831,0,0
142,L18 DC3 GTI,29788,29790,0,0


In [86]:
# LANTAI 3

time_start = "2023-11-30 22:00:00"
time_till = "2023-11-30 23:59:59"


base_dir = './result/Lantai 3/DCI - Lantai 3 (Nov 2023)/'

timestamp_list = pd.date_range(start=time_start, end=time_till, freq='1T')
# summary = pd.DataFrame(columns=['hostname','total_temp', 'latest_temp_timestamp','total_hum','latest_hum_timestamp'])
summary = pd.DataFrame(columns=['hostname','temp_id', 'total_temp','hum_id','total_hum'])

# for index, data in df_rekap[:-1].iterrows():
for index, data in df_rekap_3.iterrows():    
    hostname = data['hostname']
    host_id = ['']
    hum_id = data['hum_id']
    temp_id = data['temp_id']

    raw_temp = pd.DataFrame(columns=['itemid','clock','value','ns'])
    raw_hum = pd.DataFrame(columns=['itemid','clock','value','ns'])

    raw_temp_data = get_data(auth, data['temp_hist'], time_start, time_till, temp_id)
    raw_hum_data = get_data(auth, data['hum_hist'],time_start, time_till, hum_id)


    raw_temp = pd.concat([raw_temp, raw_temp_data], axis=0)
    raw_hum = pd.concat([raw_hum, raw_hum_data], axis=0)
    raw_temp['date'] = raw_temp['clock'].apply(to_timestamp)
    raw_hum['date'] = raw_hum['clock'].apply(to_timestamp)

    raw_temp = raw_temp.groupby(by='date').max().reset_index()
    raw_hum = raw_hum.groupby(by='date').max().reset_index()


    result = pd.DataFrame(columns=['hostname','date'])
    result['date'] = timestamp_list
    result['date'] = result['date'].astype('string')
    # result['date'] = pd.concat([raw_temp['date'], raw_hum['date']]).sort_values(ascending=True).unique()
    result = pd.merge(result, raw_temp[['date','value']].rename(columns={'value':'temp'}), on=['date'], how='left')
    result = pd.merge(result, raw_hum[['date','value']].rename(columns={'value':'hum'}), on=['date'], how='left')

    result['hostname'] = hostname
    # result[['temp','hum']] = result[['temp','hum']].astype('float')
    result[['temp_null','hum_null']] = ''
    
    final_data = fill_null(result)
    # final_data.to_csv(str("./result/DCI - Lantai 3 (Nov 2023)/"+hostname+'.csv'), index=False)
    data = {'hostname': hostname,
        'temp_id': temp_id,
        'total_temp':len(raw_temp),
        'hum_id': hum_id,
        'total_hum': len(raw_hum),
    }
    summary.loc[len(summary)] = data
    print(hostname)
    

summary.to_excel(str(base_dir+'summary4.xlsx'), index=False)

      hostname  temp_id  total_temp  hum_id  total_hum
0  E10 DC3 GTI    30440          24   30438         60
      hostname  temp_id  total_temp  hum_id  total_hum
0  E10 DC3 GTI    30440          24   30438         60
1  E13 DC3 GTI    30458          24   30456         60
      hostname  temp_id  total_temp  hum_id  total_hum
0  E10 DC3 GTI    30440          24   30438         60
1  E13 DC3 GTI    30458          24   30456         60
2  E18 DC3 GTI    30288          24   30286         60
      hostname  temp_id  total_temp  hum_id  total_hum
0  E10 DC3 GTI    30440          24   30438         60
1  E13 DC3 GTI    30458          24   30456         60
2  E18 DC3 GTI    30288          24   30286         60
3  E26_DC3_GTI    31669          24   31667         60
      hostname  temp_id  total_temp  hum_id  total_hum
0  E10 DC3 GTI    30440          24   30438         60
1  E13 DC3 GTI    30458          24   30456         60
2  E18 DC3 GTI    30288          24   30286         60
3  E26_DC3

In [72]:
result

,hostname,date,temp,hum,temp_null,hum_null
0,E13 DC3 GTI,2023-10-31 22:00:00,20.2000,41.0000,null_value,null_value
1,E13 DC3 GTI,2023-10-31 22:01:00,20.2000,41.0000,null_value,
2,E13 DC3 GTI,2023-10-31 22:02:00,20.2000,41.0000,,null_value
3,E13 DC3 GTI,2023-10-31 22:03:00,20.2000,42.0000,null_value,
4,E13 DC3 GTI,2023-10-31 22:04:00,20.2000,42.0000,null_value,null_value
...,...,...,...,...,...,...
115,E13 DC3 GTI,2023-10-31 23:55:00,20.0000,41.0000,null_value,
116,E13 DC3 GTI,2023-10-31 23:56:00,20.0000,41.0000,null_value,null_value
117,E13 DC3 GTI,2023-10-31 23:57:00,20.2000,42.0000,,
118,E13 DC3 GTI,2023-10-31 23:58:00,20.2000,42.0000,null_value,null_value


In [213]:
time_start = "2023-10-01 00:00:00"
time_till = "2023-10-31 23:59:59"

hostname = 'E10 DC3 GTI'
hum_id = 30438
temp_id = 30440

raw_temp = pd.DataFrame(columns=['itemid','clock','value','ns'])
raw_hum = pd.DataFrame(columns=['itemid','clock','value','ns'])

raw_temp_data = get_data(auth, time_start, time_till, temp_id)
raw_hum_data = get_data(auth, time_start, time_till, hum_id)


raw_temp = pd.concat([raw_temp, raw_temp_data], axis=0)
raw_hum = pd.concat([raw_hum, raw_hum_data], axis=0)
raw_temp['date'] = raw_temp['clock'].apply(to_timestamp)
raw_hum['date'] = raw_hum['clock'].apply(to_timestamp)

raw_temp = raw_temp.groupby(by='date').max().reset_index()
raw_hum = raw_hum.groupby(by='date').max().reset_index()


result = pd.DataFrame(columns=['hostname','hostid','date','temp_null','hum_null'])
result['date'] = pd.concat([raw_temp['date'], raw_hum['date']]).sort_values(ascending=True).unique()
result = pd.merge(result, raw_temp[['date','value']].rename(columns={'value':'temp'}), on=['date'], how='left')
result = pd.merge(result, raw_hum[['date','value']].rename(columns={'value':'hum'}), on=['date'], how='left')

result['hostname'] = hostname
result[['temp','hum']] = result[['temp','hum']].astype('float')

result.to_csv(str("./test_data/"+hostname+'_old'+'.csv'), index=False)

In [8]:
summary

,hostname,total_temp,latest_temp_timestamp,total_hum,latest_hum_timestamp
0,GD41.G21 SmartPDU B,120,2023-11-30 00:00:00,120,2023-11-30 00:00:00
1,GD41.G21 SmartPDU C,120,2023-11-30 00:00:00,120,2023-11-30 00:00:00
2,GD41.G21 SmartPDU D,120,2023-11-30 00:00:00,120,2023-11-30 00:00:00
3,GD41.G24 SmartPDU A,120,2023-11-30 00:00:00,120,2023-11-30 00:00:00
4,GD41.G24 SmartPDU B,120,2023-11-30 00:00:00,120,2023-11-30 00:00:00
...,...,...,...,...,...
94,GD41.J62,0,1970-01-01 07:00:00,0,1970-01-01 07:00:00
95,GD41.K59,0,1970-01-01 07:00:00,0,1970-01-01 07:00:00
96,GD41.K62,0,1970-01-01 07:00:00,0,1970-01-01 07:00:00
97,GD41.L62,0,1970-01-01 07:00:00,0,1970-01-01 07:00:00


In [28]:
dataframe = result

In [227]:
fill_null(dataframe)

KeyError: 44640

In [39]:
# for i in range(len(dataframe)):
#     for column in dataframe[['temp','hum']]:
#         if pd.isnull(dataframe.at[i, column]):
#             if i == 0:
#                 num = 0
#                 while pd.isnull(dataframe.at[num + 1, 'temp']) and num != len(dataframe):                    
#                     num +=1
#                 next_value = dataframe.at[num +1, column]
#                 print(num)
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = next_value
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
#             else:
#                 prev_value = dataframe.at[i - 1, column]
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = prev_value
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'

for i in range(len(dataframe)):
    for column in dataframe[['temp','hum']]:
        if pd.isnull(dataframe.at[i, column]):
            if i == 0:
                num = 0
                while num < len(dataframe)-1 and pd.isnull(dataframe.at[num, column]):                    
                    num +=1
                next_value = dataframe.at[num, column]
                dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = next_value
                dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
            else:
                prev_value = dataframe.at[i - 1, column]
                dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = prev_value
                dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'




# for i in range(len(dataframe)):
#     for column in dataframe[['temp','hum']]:
#         if pd.isnull(dataframe.at[i, column]):
#             if i == 0:
#                 num = 0
#                 while pd.isnull(dataframe.at[num, column]):                    
#                     num +=1
#                     print(num)
#                 next_value = dataframe.at[num +1, column]
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = next_value
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'
#             else:
#                 prev_value = dataframe.at[i - 1, column]
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column) ]] = prev_value
#                 dataframe.iloc[i, [dataframe.columns.get_loc(column+'_null') ]] = 'null_value'                

In [42]:
dataframe

,hostname,date,temp,hum,temp_null,hum_null
0,GN31.D4,2023-10-01 00:00:00,18.3,NaN,,null_value
1,GN31.D4,2023-10-01 00:01:00,18.3,NaN,null_value,null_value
2,GN31.D4,2023-10-01 00:02:00,18.3,NaN,null_value,null_value
3,GN31.D4,2023-10-01 00:03:00,18.3,NaN,null_value,null_value
4,GN31.D4,2023-10-01 00:04:00,18.3,NaN,null_value,null_value
...,...,...,...,...,...,...
44635,GN31.D4,2023-10-31 23:55:00,18.4,NaN,,null_value
44636,GN31.D4,2023-10-31 23:56:00,18.4,NaN,null_value,null_value
44637,GN31.D4,2023-10-31 23:57:00,18.4,NaN,null_value,null_value
44638,GN31.D4,2023-10-31 23:58:00,18.4,NaN,null_value,null_value


In [38]:
# # len(dataframe['hum'])
# # num !=len(dataframe)

# # pd.isnull(dataframe.at[44640, 'hum'])
# num != len(dataframe)
num = 44600
len(dataframe)
while num < len(dataframe)-1 and pd.isnull(dataframe.at[num, column]):
    num +=1
num
# num < len(dataframe)
# num < len(dataframe) and (pd.isnull(dataframe.at[num, column])):                 
#     num +=1
#     print(num)

44639

In [228]:
dataframe

,hostname,date,temp,hum,temp_null,hum_null
0,GN31.D4,2023-10-01 00:00:00,18.3,NaN,,null_value
1,GN31.D4,2023-10-01 00:01:00,18.3,NaN,null_value,null_value
2,GN31.D4,2023-10-01 00:02:00,18.3,NaN,null_value,null_value
3,GN31.D4,2023-10-01 00:03:00,18.3,NaN,null_value,null_value
4,GN31.D4,2023-10-01 00:04:00,18.3,NaN,null_value,null_value
...,...,...,...,...,...,...
44635,GN31.D4,2023-10-31 23:55:00,18.4,NaN,,null_value
44636,GN31.D4,2023-10-31 23:56:00,18.4,NaN,null_value,null_value
44637,GN31.D4,2023-10-31 23:57:00,18.4,NaN,null_value,null_value
44638,GN31.D4,2023-10-31 23:58:00,18.4,NaN,null_value,null_value


In [88]:
raw_temp2 = pd.DataFrame(columns=['itemid','clock','value','ns'])
raw_hum2 = pd.DataFrame(columns=['itemid','clock','value','ns'])
raw_temp2 = pd.concat([raw_temp, raw_temp2], axis=0)
raw_hum2 = pd.concat([raw_hum, raw_hum2], axis=0)
raw_temp2['date'] = raw_temp2['clock'].apply(to_timestamp)
raw_hum2['date'] = raw_hum2['clock'].apply(to_timestamp)

raw_temp2 = raw_temp2.groupby(by='date').max().reset_index()
raw_hum2 = raw_temp2.groupby(by='date').max().reset_index()


test = pd.DataFrame(columns=['hostname','hostid','date','temp','hum'])
test['date'] = pd.concat([raw_temp2['date'], raw_hum2['date']]).sort_values(ascending=True).unique()

In [92]:
test = pd.DataFrame(columns=['hostname','hostid','date'])
test['date'] = pd.concat([raw_temp2['date'], raw_hum2['date']]).sort_values(ascending=True).unique()

test = pd.merge(test, raw_hum2[['date','value']].rename(columns={'value':'hum'}), on=['date'], how='left')
test = pd.merge(test, raw_temp2[['date','value']].rename(columns={'value':'temp'}), on=['date'], how='left')


In [115]:
result = pd.DataFrame(columns=['hostname','hostid','date'])
result['date'] = pd.concat([raw_temp['date'], raw_hum['date']]).sort_values(ascending=True).unique()
result = pd.merge(result, raw_temp[['date','value']].rename(columns={'value':'temp'}), on=['date'], how='left')
result = pd.merge(result, raw_hum[['date','value']].rename(columns={'value':'hum'}), on=['date'], how='left')
result[['temp','hum']] = result[['temp','hum']].astype('float')

,temp,hum
0,17.3,46.0
1,NaN,47.0
2,NaN,48.0
3,16.5,NaN
4,NaN,48.0
...,...,...
67,NaN,46.0
68,NaN,47.0
69,16.7,NaN
70,NaN,48.0


,temp,hum
0,17.3,46.0
1,NaN,47.0
2,NaN,48.0
3,16.5,NaN
4,NaN,48.0
...,...,...
67,NaN,46.0
68,NaN,47.0
69,16.7,NaN
70,NaN,48.0


In [47]:
raw_temp.to_excel('./test_data/J42 DC3 GTI.xlsx', index=False)
summary

,hostname,total_temp,latest_temp_clock,total_hum,latest_hum_clock,latest_temp_timestamp,latest_hum_timestamp
0,J42 DC3 GTI,22295,1696093218,22295,1696093216,2023-10-01 00:00:18,2023-10-01 00:00:16


In [29]:
# len(raw_temp)
"I42 DC3 GTI intervalnya 5 menit" 

8918

In [43]:
# Zabbix Host	Host ID	Zabbix Item	itemid
# GTI_GU21_UPSA1	10268	output KW	28700
# GTI_GU21_UPSA2	10269	output KW	28723
# GTI_GU21_UPSA3	10270	output KW	28746
# GTI_GU21_UPSB1	10267	output KW	28677
# GTI_GU21_UPSB2	10271	output KW	28768
# GTI_GU21_UPSB3	10272	output KW	28790
# GTI_GU41_UPSA1	10635	output KW	33093
# GTI_GU41_UPSA2	10638	output KW	33096
# GTI_GU42_UPSB1	10636	output KW	28677 / 33094
# GTI_GU42_UPSB2	10637	output KW	28768 / 33095

# GTI_GU21_UPSA1 = get_data(auth, time_start, time_till, 28700, 10268)
# GTI_GU21_UPSA2 = get_data(auth, time_start, time_till, 28723, 10269)
# GTI_GU21_UPSA3 = get_data(auth, time_start, time_till, 28746, 10270)
# GTI_GU21_UPSB1 = get_data(auth, time_start, time_till, 28677, 10267)
# GTI_GU21_UPSB2 = get_data(auth, time_start, time_till, 28768, 10271)
# GTI_GU21_UPSB3 = get_data(auth, time_start, time_till, 28790, 10272)
# GTI_GU41_UPSA1 = get_data(auth, time_start, time_till, 33093, 10635)
# GTI_GU41_UPSA2 = get_data(auth, time_start, time_till, 33096, 10638)
GTI_GU41_UPSB1 = get_data(auth, time_start, time_till, 28677, 10636)
GTI_GU41_UPSB2 = get_data(auth, time_start, time_till, 28768, 10637)


In [41]:
GTI_GU21_UPSA1['value']

0       56499.0000
1       56499.0000
2       56499.0000
3       56499.0000
4       56499.0000
           ...    
8635    56698.0000
8636    56698.0000
8637    56299.0000
8638    56299.0000
8639    56099.0000
Name: value, Length: 8640, dtype: object

In [42]:
GTI_GU21_UPSA1.to_csv('GTI_GU21_UPSA1.csv', index=False)
GTI_GU21_UPSA2.to_csv('GTI_GU21_UPSA2.csv', index=False)
GTI_GU21_UPSA3.to_csv('GTI_GU21_UPSA3.csv', index=False)
GTI_GU21_UPSB1.to_csv('GTI_GU21_UPSB1.csv', index=False)
GTI_GU21_UPSB2.to_csv('GTI_GU21_UPSB2.csv', index=False)
GTI_GU21_UPSB3.to_csv('GTI_GU21_UPSB3.csv', index=False)
GTI_GU41_UPSA1.to_csv('GTI_GU41_UPSA1.csv', index=False)
GTI_GU41_UPSA2.to_csv('GTI_GU41_UPSA2.csv', index=False)
GTI_GU41_UPSB1.to_csv('GTI_GU41_UPSB1.csv', index=False)
GTI_GU41_UPSB2.to_csv('GTI_GU41_UPSB2.csv', index=False)